In [2]:
import random 
import time
import math

# this function checks if two tasks are conflicting. It assumes L is sorted according to starting time
def is_conflict(L):
    for i in range(len(L)-1):
        if L[i][1] > L[i+1][0]: return True
    return False

# this function makes a random search for assignments
def random_search(L):
    vec_assignment = [0]*len(L)
    
    while True:         
        non_conflicting_tasks = []
        for i,el in enumerate(L):
            if vec_assignment[i] == 0:
                vec_assignment[i] = 1
                assignment = [L[k] for k in range(len(L)) if vec_assignment[k]==1 ]
                if not is_conflict(assignment):
                    non_conflicting_tasks.append(i)
                vec_assignment[i] = 0
                        
        if len(non_conflicting_tasks)==0:
            assignment = [L[k] for k in range(len(L)) if vec_assignment[k]==1 ]
            val = sum([k[2] for k in assignment])
            return (val,assignment)
        
        i = non_conflicting_tasks[random.randint(0,len(non_conflicting_tasks)-1)]
        vec_assignment[i] = 1        


# this function makes a brute force search for assignments
def brute_force(L):
    n = len(L)
    max_benefit = 0
    best_subset = None
    # Loop that enumerates all possible non-empty subsets of requests
    for i in range(1, 2**n):
        indexes = list(map(int, format(i, '0' + str(n) + 'b')))  # binary string corresponding to selected requests
        subset = [item for (item, index) in zip(L, indexes) if index]   # actual subset of requests
        if is_conflict(subset): # if subset contains conflicting requests, ignore it
            continue
        else:
            # Keep track of subsets with a better benefit 
            # than previously selected
            benefit = sum(k for i, j, k in subset)
            if benefit > max_benefit:
                max_benefit = benefit
                best_subset = subset
    return (max_benefit, best_subset)


# this function makes a greedy force search for assignments
def greedy(L):
    output = []  
    # Sort requests by decreasing benefit
    L = sorted(L, key = lambda x: x[2], reverse = True)
    # print(L)
    for request in L:
        # print(request)
        if not output:
            output.append(request)
            continue
        if output[-1][1] > request[0]: # conflict with last added request
            # For a 1 request output, check if current request
            # can be added before it without conflicts
            if len(output) < 2:
                if is_conflict([request, output[0]]):
                    continue
                else:
                    output = [request] + output
            else:
                # For output of >= 2 requests, check if we can add
                # the current request in between two requests in the output
                # starting from the beginning without conflicts. If there is
                # a conflict at the insertion point between the jth and (j + 1)th
                # request, we consider the next insertion point between the 
                # (j + 1)th and the (j + 2)th request as long as the finish time
                # of the request after the insertion point is smaller than 
                # the starting time of current request.
                if not is_conflict([request, output[0]]):    # Check if 
                # current request may be added before 1st added request
                    output = [request] + output
                    continue
                j = 0
                while j <= len(output) - 2:
                    if is_conflict([output[j], request, output[j + 1]]):
                        if output[j + 1][1] > request[0]:  
                            break
                    else: 
                        output = output[:j + 1] + [request] + output[j + 1:]
                        break
                    j += 1

        else:
            output.append(request)  # No conflict with last added request
        # print(output)

    return (sum(k for i, j, k in output), output)


# this function makes a dynamic programing search for assignments
def dynamic_prog(L):
    B = [0]*(len(L) + 1)    # Tracks maximum benefit with first i requests
    P = [0]*(len(L) + 1)    # Tracks index of latest request that does not 
    # conflict with each ith request, i = 1,2,..,n. We define P[0] = P[1] = 0.
    output = []
    # Sort requests by increasing finish times
    L = [0] + sorted(L, key = lambda x: x[1])   # Add dummy element to beginning 
                                                # for consistent indexing with 
                                                # B and P lists
    # Loop that updates the P list values
    for i in range(2, len(P)):
        j = i - 1
        while j >= 1:
            if is_conflict([L[j], L[i]]):
                j -= 1
            else:
                P[i] = j
                break

    # For each B[i], it is the maximum of B[i - 1] (we do not add
    # the ith request) and B[P[i]] + b_i (we add the ith request with
    # benefit b_i and the maximum benefit from previous requests that do
    # not conflict with the ith request). 
    for idx, request in enumerate(L[1:], start = 1):
        B[idx] = max(B[idx - 1], B[P[idx]] + request[2])

    # print(B)
    # print("\n")
    # print(P)
    # print("\n")
    # print(L)

    # To get the optimal set of requests, we backtrack 
    # from B[n]. If B[i] == B[i - 1], that means we did
    # not add the ith request, so we consider the (i - 1)th
    # request. Otherwise we added the ith request i.e. B[i] 
    # == B[P[i]] + b_i so we append it to our output and 
    # consider the (P[i])th request. Since our list of request
    # is sorted in order of increasing finish times, we return
    # a reversed version of the output, which would necessarily
    # be in sorted order of start times. 

    i = len(B) - 1
    while i >= 1:
        if B[i] == B[i - 1]:
            i -= 1
            continue
        elif B[i] == B[P[i]] + L[i][2]:
            output.append(L[i])
            i = P[i]

    return (B[-1], output[::-1])


# this function prints the taskes
def print_tasks(L):
    for i,t in enumerate(L):
        print("task %2i (b=%2i):" %(i,t[2]),end="")
        print(" "*round(t[0]/10) + "-"*round((t[1]-t[0])/10))
        

# this function tests and times a telescope tasks assignment search
def test_telescope(algo,my_tab,display):
    tab = my_tab.copy()
    print("testing",algo,str(" "*(14-len(algo))),"... ",end='')
    t = time.time()
    (max_temp,assignment_temp) = eval(algo + "(tab)")
    print("done ! It took {:.2f} seconds".format(time.time() - t))
    if max_temp!=None:
        print("Solution with benefit = %i" %(max_temp),end='\n')
    if display: 
        if assignment_temp!=None:
            print_tasks(assignment_temp)
            print()
    

MAX_BENEFIT = 99
MAX_START_TIME = 500
MAX_DURATION = 250

NUMBER_OF_ELEMENTS = 10
print("\n ******** Testing to solve for %i events ********" %(NUMBER_OF_ELEMENTS))
val = [(random.randint(1, MAX_START_TIME),random.randint(1, MAX_DURATION),random.randint(1, MAX_BENEFIT)) for i in range(NUMBER_OF_ELEMENTS)] 
tab = sorted([(val[i][0],val[i][0]+val[i][1],val[i][2]) for i in range(NUMBER_OF_ELEMENTS)])
print("Problem instance: ")
print_tasks(tab)
print("")
test_telescope("random_search",tab,True)
test_telescope("brute_force",tab,True)
test_telescope("greedy",tab,True)
test_telescope("dynamic_prog",tab,True)


NUMBER_OF_ELEMENTS = 20
print("\n ******** Testing to solve for %i events ********" %(NUMBER_OF_ELEMENTS))
val = [(random.randint(1, MAX_START_TIME),random.randint(1, MAX_DURATION),random.randint(1, MAX_BENEFIT)) for i in range(NUMBER_OF_ELEMENTS)] 
tab = sorted([(val[i][0],val[i][0]+val[i][1],val[i][2]) for i in range(NUMBER_OF_ELEMENTS)])
test_telescope("random_search",tab,False)
test_telescope("brute_force",tab,False)
test_telescope("greedy",tab,False)
test_telescope("dynamic_prog",tab,False)


 ******** Testing to solve for 10 events ********
Problem instance: 
task  0 (b=72):  ------------
task  1 (b=70):    -------------------
task  2 (b=53):       --------
task  3 (b=72):                -------------
task  4 (b=59):                        ------------------------
task  5 (b=32):                              -------------------------
task  6 (b=12):                                  ---------------
task  7 (b=68):                                     ------------
task  8 (b= 9):                                                --------
task  9 (b=99):                                                --------------

testing random_search   ... done ! It took 0.00 seconds
Solution with benefit = 138
task  0 (b=70):    -------------------
task  1 (b=59):                        ------------------------
task  2 (b= 9):                                                --------

testing brute_force     ... done ! It took 0.00 seconds
Solution with benefit = 243
task  0 (b=72):  --------